---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from sklearn import preprocessing
#import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [11]:
def blight_model():
    
    # read training data file and combine with location data into a full set of training data
    # with ticket_id as the index
    train_raw = pd.read_csv('train.csv',encoding='latin1',low_memory=False)
    latlons = pd.read_csv('latlons.csv', encoding='latin1')
    addresses = pd.read_csv('addresses.csv', encoding='latin1')
    
    # latlons maps address to coordinates and addresses maps ticket_id to address
    loc = addresses.merge(latlons, how='inner', on='address')
    train_full = train_raw.merge(loc, how='inner', on='ticket_id').set_index('ticket_id')
    
    #print(train_full.columns)
    
    # remove columns that leak data or hold info for target var
    # these features only in training data
    train_set = train_full.drop(['payment_amount','balance_due','payment_date','payment_status',
                                 'collection_status','compliance_detail'],axis=1)
    
    # feature selection
    # encode string type features to include them in correlation heatmap
    
    # drop features that don't make sense to encode
    # drop more after inspection of correlation heatmap
    # to be dropped from final test data as well
    drop_features = ['inspector_name','violator_name','violation_street_number','violation_street_name',
                           'violation_zip_code','mailing_address_str_number','mailing_address_str_name',
                            'non_us_str_code','hearing_date','violation_description','admin_fee','state_fee',
                            'late_fee','discount_amount','clean_up_cost','address',
                           'agency_name','state','country','ticket_issued_date']
    
    train = train_set.drop(drop_features,axis=1).fillna(method='pad')
    
    train['grafitti_status'].fillna('',inplace=True)
        
    # separate target variable from feature data
    X = train.drop(['compliance'],axis=1)
    y = train['compliance']
    
    le = preprocessing.LabelEncoder()
    
    for feature in X.columns:
        if isinstance(feature, str):
            
            #print(feature)
            le.fit(X[feature])
            X[feature] = le.transform(X[feature])
            
    # create correlation heatmap
    #plt.figure(figsize=(10,10))
    #correlation = X.corr()
    #sns.heatmap(correlation, annot=True)
    #plt.xticks(rotation=45)
    #plt.yticks(rotation=45)
    
    #plt.show()
    
    #print(train_set.columns)
    #print(correlation)
    
    # scale and split training data
    # use train test split on training data, save test data for final validation
    scaler = preprocessing.MinMaxScaler()
    X = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)
    
#######################################################################################################################
    
    # try a few different models and compare accuracy scores
    #models = []
    #models.append(('K Neighbors', KNeighborsClassifier()))
    #models.append(('Logistic Regression', LogisticRegression(solver='liblinear', multi_class='ovr')))
    #models.append(('Decision Tree', DecisionTreeClassifier()))
    
    # use stratified k-fold splits for model selection
    #output = []
    #names = []
    #for name, model in models:
        #kfold = StratifiedKFold(n_splits=3, random_state=0, shuffle=True)
        #cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
        #output.append(cv_results)
        #names.append(name)
        #print('%s\n Mean Score: %f\n' % (name, cv_results.mean()) )
        
    # use classifier with highest average cross validation score (logistic regression)
    # perform gridsearch to tune parameters
    #lr_grid = LogisticRegression(solver='liblinear', multi_class='ovr')
    #grid_values = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100,1000]}
    #gridsearch = GridSearchCV(lr_grid, grid_values, scoring='roc_auc')
    
    #model = gridsearch.fit(X_train, y_train)
    #best_params = model.best_estimator_.get_params()
    
    # now we have the best performing parameters and we can evaluate using accuracy score
    #lr_clf = LogisticRegression(**best_params).fit(X_train,y_train)
    #pred = lr_clf.predict(X_test)
    #print(accuracy_score(y_test, pred))
    
    # have a good accuracy score, now repeat steps to run test data through LR classifier
    
#######################################################################################################################
    
    # load test data
    test_raw = pd.read_csv('test.csv',encoding='latin1',low_memory=False)
    loc = addresses.merge(latlons, how='inner', on='address')
    test_full = test_raw.merge(loc, how='inner', on='ticket_id').set_index('ticket_id')
    test = test_full.drop(drop_features,axis=1).fillna(method='pad')
    test['grafitti_status'].fillna('',inplace=True)
    
      
    for feature in test.columns:
        if isinstance(feature, str):
            le.fit(test[feature])
            test[feature] = le.transform(test[feature])
            
    X_test_final = scaler.fit_transform(test)
    
#######################################################################################################################
    
    # now create the series of probability values for each ticket
    #pred_prob = lr_clf.predict_proba(X_test_final)[:,1]
    #prob = pd.Series(pred_prob,index=test.index)
    
#######################################################################################################################
    
    # Logistic Regression classifier performed poorly when evaluated by AUC
    # repeat previous model selection steps with classidier models in sklearn ensemble package
    

    
    ensemble_models = []
    #ensemble_models.append(('Random Forest', RandomForestClassifier()))
    #ensemble_models.append(('Gradient Boosting', GradientBoostingClassifier()))
    ensemble_models.append(('ADA Booster', AdaBoostClassifier()))
    
    # use stratified k-fold splits for model selection
    output = []
    names = []
    for name, model in ensemble_models:
        kfold = StratifiedKFold(n_splits=3, random_state=0, shuffle=True)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='roc_auc')
        output.append(cv_results)
        names.append(name)
        #print('%s\n Mean Score: %f\n' % (name, cv_results.mean()) )
        
    # use classifier with highest average cross validation score (ada booster)
    # perform gridsearch to tune parameters
    ada_grid = AdaBoostClassifier()
    grid_values = {'learning_rate': [.5,.75,1,1.25,1.5,1.75,2]}
    
    gridsearch = GridSearchCV(ada_grid, grid_values, scoring='roc_auc')
    
    model = gridsearch.fit(X_train, y_train)
    best_params = model.best_estimator_.get_params()
    #print(best_params)
    
    # now we have the best performing parameters and we can evaluate using accuracy score
    adaboost_clf = AdaBoostClassifier(**best_params).fit(X_train,y_train)
    pred = adaboost_clf.predict(X_test)
    #print(accuracy_score(y_test, pred))
    
    # now create the series of probability values for each ticket
    pred_prob = adaboost_clf.predict_proba(X_test_final)[:,1]
    prob = pd.Series(pred_prob,index=test.index)
    
    return prob
blight_model()

ticket_id
284932    0.491861
285362    0.487822
285361    0.492692
285338    0.492734
285346    0.493527
285345    0.492734
285347    0.490195
285342    0.499892
285530    0.488657
284989    0.489393
285344    0.490963
285343    0.489425
285340    0.488657
285341    0.490195
285349    0.494347
285348    0.493553
284991    0.490967
285532    0.490148
285406    0.489313
285001    0.489331
285006    0.489399
285405    0.487068
285337    0.488580
285496    0.489681
285497    0.492220
285378    0.487068
285589    0.490132
285585    0.491899
285501    0.493014
285581    0.487089
            ...   
376367    0.488464
376366    0.490340
376362    0.497497
376363    0.499056
376365    0.488464
376364    0.490340
376228    0.491206
376265    0.490688
376286    0.494190
376320    0.490662
376314    0.489882
376327    0.495767
376385    0.494152
376435    0.497197
376370    0.499074
376434    0.490102
376459    0.491628
376478    0.484196
376473    0.489882
376484    0.489567
376482    0.490083
37

In [10]:
#print(blight_model().size)

In [28]:
# !ls

In [21]:
# !cat readonly/test.csv > test.csv

In [29]:
# !ls

In [25]:
# !cat readonly/latlons.csv > latlons.csv
# !cat readonly/addresses.csv > addresses.csv

In [1]:
# !ls

#  Load Training Data

# read training data file and combine with location data into a full set of training data
# with ticket_id as the index
train_raw = pd.read_csv('train.csv',encoding='latin1',low_memory=False)
latlons = pd.read_csv('latlons.csv', encoding='latin1')
addresses = pd.read_csv('addresses.csv', encoding='latin1')
    
# latlons maps address to coordinates and addresses maps ticket_id to address
loc = addresses.merge(latlons, how='inner', on='address')
train_full = train_raw.merge(loc, how='inner', on='ticket_id').set_index('ticket_id')
    
#print(train_full.columns)
    
# remove columns that leak data or hold info for target var
# these features only in training data
train_set = train_full.drop(['payment_amount','balance_due','payment_date','payment_status',
                                 'collection_status','compliance_detail'],axis=1)
#train_set.head()

# Feature Selection

# drop features that don't make sense to encode
# drop more after inspection of correlation heatmap
# to be dropped from final test data as well
drop_features = ['inspector_name','violator_name','violation_street_number','violation_street_name',
                 'violation_zip_code','mailing_address_str_number','mailing_address_str_name',
                 'non_us_str_code','hearing_date','violation_description','admin_fee','state_fee',
                 'late_fee','discount_amount','clean_up_cost','address',
                 'agency_name','state','country','ticket_issued_date']
    
train = train_set.drop(drop_features,axis=1).fillna(method='pad')
    
train['grafitti_status'].fillna('',inplace=True)

# separate target variable from feature data
X = train.drop(['compliance'],axis=1)
y = train['compliance']

#print(X.head())

le = preprocessing.LabelEncoder()
    
for feature in X.columns:
    if isinstance(feature, str):
            
        #print(feature)
        le.fit(X[feature])
        X[feature] = le.transform(X[feature])

# scale and split training data
# use train test split on training data, save test data for final validation
scaler = preprocessing.MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

#print(X)

# Model Selection

# Logistic Regression classifier performed poorly when evaluated by AUC
# repeat previous model selection steps with classidier models in sklearn ensemble package
    
ensemble_models = []
ensemble_models.append(('Random Forest', RandomForestClassifier()))
ensemble_models.append(('Gradient Boosting', GradientBoostingClassifier()))
ensemble_models.append(('ADA Booster', AdaBoostClassifier()))
    
# use stratified k-fold splits for model selection
output = []
names = []
for name, model in ensemble_models:
    kfold = StratifiedKFold(n_splits=3, random_state=0, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='roc_auc')
    output.append(cv_results)
    names.append(name)
    print('%s\n Mean Score: %f\n' % (name, cv_results.mean()) )

# Tune Parameters

# use classifier with highest average cross validation score (ada booster)
# perform gridsearch to tune parameters
ada_grid = AdaBoostClassifier()
grid_values = {'learning_rate': [.5,.75,1,1.25,1.5,1.75,2]}
    
gridsearch = GridSearchCV(ada_grid, grid_values)
    
model = gridsearch.fit(X_train, y_train)
best_params = model.best_estimator_.get_params()
print(best_params)
    
# now we have the best performing parameters and we can evaluate using accuracy score
adaboost_clf = AdaBoostClassifier(**best_params).fit(X_train,y_train)
pred = adaboost_clf.predict(X_test)
print(accuracy_score(y_test, pred))

# Load Final Test Data

test_raw = pd.read_csv('test.csv',encoding='latin1',low_memory=False)
loc = addresses.merge(latlons, how='inner', on='address')
test_full = test_raw.merge(loc, how='inner', on='ticket_id').set_index('ticket_id')
test = test_full.drop(drop_features,axis=1).fillna(method='pad')
test['grafitti_status'].fillna('',inplace=True)

#print(test.head())

for feature in test.columns:
    if isinstance(feature, str):
        le.fit(test[feature])
        test[feature] = le.transform(test[feature])
            
X_test_final = scaler.fit_transform(test)
#X_test_final

# now create the series of probability values for each ticket
pred_prob = adaboost_clf.predict_proba(X_test_final)[:,0]
print(np.max(pred_prob))